# Proyecto: Creación de Data Frames a partir de RDDs

En esta practica veremos como crear un Data Frame a partir de la unión de 2 RDDs,

para ello usaremos datos de 2 archivos .csv de los cuales, uno tiene encabezado y el otro no, 

así que realizaremos los siguientes pasos:

1. Creamos los RDDs a partir de archivos.csv

2. Eliminamos encabezdo del RDD que trae encabezado

3. Unimos los RDDs en uno solo

4. Tranfomramos el RDD en un Data Frame

In [1]:
# libreria para crear punto de conexión:
from pyspark import SparkContext

# Cargamos libreria para crear Data Frames:
from pyspark.sql import SQLContext

#from pyspark.sql import SparkSession

# Cargamos librerias para crear el schema del DataFrame
from pyspark.sql.types import StructType, StructField

# Cargamos los tipos de datos que usaremos para crear las columnas de los dataframes:
from pyspark.sql.types import  IntegerType, StringType, FloatType

#from pyspark.sql.types import Row

In [2]:
# Creamos el punto de conexió a Spark (que se ejecutará en mi máquina 'local'):
spark = SparkContext(master='local', appName='RDDs_to_DataFrames')

In [3]:
# Creamos el contexto para SQL:
sqlContext = SQLContext(spark)

/home/marck/.local/lib/python3.8/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


## Exploramos contenido de archivos `.csv`

In [4]:
# Visualizamos el contenido de la carpeta 'Data' 
!ls ./Data/

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [5]:
# Ruta en donde se encuentran los datos con los que trabajaremos:
path = './Data/'

In [6]:
# Vemos las primeras 5 líneas del archivo 'deportista.csv'

!head -n 5 Data/deportista.csv

# Veremos que el archivo ya tiene un encabezado

deportista_id,nombre,genero,edad,altura,peso,equipo_id
1,A Dijiang,1,24,180,80,199
2,A Lamusi,1,23,170,60,199
3,Gunnar Nielsen Aaby,1,24,0,0,273
4,Edgar Lindenau Aabye,1,34,0,0,278


In [7]:
# Vemos las primeras 5 líneas del archivo 'deportista2.csv'

!head -n 5 Data/deportista2.csv

# Veremos que el archivo No tiene un encabezado

67787,Lee BongJu,1,27,167,56,970
67788,Lee BuTi,1,23,164,54,203
67789,Anthony N. Buddy Lee,1,34,172,62,1096
67790,Alfred A. Butch Lee Porter,1,19,186,80,825
67791,Lee ByeongGu,1,22,175,68,970


Notemos que el archivo 'deportista.csv' tiene encabezado y 'deportista2.csv' no tiene

El archivo 'deportista2.csv' es continuación de 'deportista.csv'

Para hacer un sólo Data Frame, haremos lo siguiente: 

1. Tranformamos los archivos.csv a RDDs

2. Eliminamos el encabezado del archivo 'deportista.csv'

3. Unimos los RDDs 'deportista' y 'deportista2

4. Ya que tenemos un sólo RDD lo pasamos a un Data Frame

## Paso 1: Creamos RDD con datos de archivos `.csv`

In [8]:
# Creamos RDDs con datos de archivos .csv
# .map(lambda l : l.split(',')) <-- Le asigan formato al contenido de archivos .csv

deportistaOlimpicoRDD = spark.textFile(path+'deportista.csv').map(lambda l : l.split(','))

deportistaOlimpicoRDD2 = spark.textFile(path+'deportista2.csv').map(lambda l : l.split(','))

In [9]:
# Número de registros a visualizar:
N=5

In [10]:
# Vemos los primeros N registros del RDD:
deportistaOlimpicoRDD.take(N)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

In [11]:
# Vemos los primeros N registros del RDD:
deportistaOlimpicoRDD2.take(N)

[['67787', 'Lee BongJu', '1', '27', '167', '56', '970'],
 ['67788', 'Lee BuTi', '1', '23', '164', '54', '203'],
 ['67789', 'Anthony N. Buddy Lee', '1', '34', '172', '62', '1096'],
 ['67790', 'Alfred A. Butch Lee Porter', '1', '19', '186', '80', '825'],
 ['67791', 'Lee ByeongGu', '1', '22', '175', '68', '970']]

## Paso 2: Eliminamos encabezado del primer RDD

In [12]:
# Construimos una función para eliminar encabezados de un RDD:

def eliminaEncabezado(indice , interador):
    return iter( list(interador)[1:] )

In [13]:
# Eliminamos encabezado al primer RDD :
deportistaOlimpicoRDD = deportistaOlimpicoRDD.mapPartitionsWithIndex(eliminaEncabezado)

# Vemos los primeros N registros del RDD:
deportistaOlimpicoRDD.take(N)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278'],
 ['5', 'Christine Jacoba Aaftink', '2', '21', '185', '82', '705']]

## Paso 3: Hacemos la unión de RDDs (sin encabezado)

In [14]:
# Conteo de registros del primer RDD:
# limite de tiempo de cálculo: 20 miliseconds
deportistaOlimpicoRDD.countApprox(20)

67786

In [15]:
# Conteo de registros del segundo RDD:
# limite de tiempo de cálculo: 20 miliseconds
deportistaOlimpicoRDD2.countApprox(20)

67785

In [16]:
# Unimos los 2 RDDs: deportistaOlimpicoRDD y deportistaOlimpicoRDD2
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union( deportistaOlimpicoRDD2 )

In [17]:
# Conteo de registros de la unión de RDDs:
# limite de tiempo de cálculo: 20 miliseconds
deportistaOlimpicoRDD.countApprox(20)

135571

## Paso 4: Tranformamos el RDD en un Data Frame

In [18]:
# Creación de un Data Frame a partir de un RDD:

# Corregimos los tipos de dato en cada columna del RDD: 
deportistaOlimpicoRDD = \
    deportistaOlimpicoRDD.map(lambda x: ( int(x[0]), x[1],  int(x[2]), int(x[3]), int(x[4]), float(x[5]), int(x[6])  ) )


# A continuación creamos el schema, es decir, la estructura (columnas) del Data Frame donde pondremos los datos del RDD:
schema = StructType([
    StructField('deportista_id', IntegerType(), False ),
    StructField('nombre', StringType(), False ),
    StructField('genero', IntegerType(), False ),
    StructField('edad', IntegerType(), False ),
    StructField('altura', IntegerType(), False ),
    StructField('peso', FloatType(), False ),
    StructField('equipo_id', IntegerType(), False )
    ])

# StructField <-- instrucción para crear una columna, se debe especificar el tipo de dato que almacenará
# En caso de que el campo pueda ser nulo ponemos 'True' de lo contrario ponemos 'False'


# Creamos el DataFrame a partir del RDD:
deportistaOlimpicoDF = sqlContext.createDataFrame(deportistaOlimpicoRDD, schema)

In [19]:
# La información de DataFrames se muestra con '.show()'
# La información de RDDs se muestra con '.take()'
deportistaOlimpicoDF.show(N)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



In [20]:
# Cerramos sesión para liberar memoria:
spark.stop()